In [1]:
import pandas as pd
import ast
import pytz
from datetime import datetime
from dateutil import tz

In [2]:
input_path ='C:/Users/level/Desktop/GROWER/outage-data-scraper/data/per_outage_Southwestern Public Service Co..csv'

In [3]:
df = pd.read_csv(input_path,low_memory=False)
df

,OBJECTID,BEGINTIME,ESTIMATEDTIMERESTORATION,CUSTOMERSOUT,EVENTSTATUS,TIMESTAMP,BATCHID,COMMUNITIES,STATES,CREWSTATUS,x,y,timestamp,EMC
0,1,2023-04-12 21:44:00,2023-04-13 00:30:00,63,Crews are working to restore power as quickly ...,NaN,12220107,Monticello,MN,NaN,-1.044018e+07,5.668449e+06,04-13-2023 04:05:16,Southwestern Public Service Co.
1,2,2023-03-28 08:39:00,NaN,2,Crews are working to restore power as quickly ...,NaN,12220107,Denver,CO,NaN,-1.168537e+07,4.834477e+06,04-13-2023 04:05:16,Southwestern Public Service Co.
2,3,2023-04-12 22:27:00,2023-04-13 01:15:00,1,The outage has been reported. Please continue ...,NaN,12220107,Brooklyn Park,MN,NaN,-1.039488e+07,5.645050e+06,04-13-2023 04:05:16,Southwestern Public Service Co.
3,4,2023-04-12 22:28:00,2023-04-13 01:15:00,1,The outage has been reported. Please continue ...,NaN,12220107,Champlin,MN,NaN,-1.039607e+07,5.649363e+06,04-13-2023 04:05:16,Southwestern Public Service Co.
4,5,2023-04-12 22:29:00,2023-04-13 01:15:00,1,The outage has been reported. Please continue ...,NaN,12220107,Minneapolis,MN,NaN,-1.038512e+07,5.624017e+06,04-13-2023 04:05:16,Southwestern Public Service Co.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002193,26,2023-10-27 19:20:00,2023-10-27 22:30:00,1,The outage has been reported. Please continue ...,NaN,27182211,Brooklyn Park,MN,NaN,-1.039033e+07,5.635733e+06,10-28-2023 00:29:48,Southwestern Public Service Co.
1002194,27,2023-10-27 18:47:00,2023-10-27 22:00:00,1,The outage has been reported. Please continue ...,NaN,27182211,Sioux Falls,SD,NaN,-1.077374e+07,5.395259e+06,10-28-2023 00:29:48,Southwestern Public Service Co.
1002195,28,2023-10-27 19:07:00,2023-10-27 20:45:00,1,Crews are working to restore power as quickly ...,NaN,27182211,Stratford,TX,NaN,-1.136269e+07,4.346929e+06,10-28-2023 00:29:48,Southwestern Public Service Co.
1002196,29,2023-10-02 10:10:00,NaN,1,Crews are working to restore power as quickly ...,NaN,27182211,Lubbock,TX,NaN,-1.135074e+07,3.972240e+06,10-28-2023 00:29:48,Southwestern Public Service Co.


In [27]:
#Layout tx6
def compute(grouped):
  eastern = tz.gettz('US/Eastern')


  grouped['timestamp'] = pd.to_datetime(grouped['timestamp'], utc= False).dt.tz_convert(eastern)
  grouped['StartDate'] = pd.to_datetime(grouped['BEGINTIME'], utc= False).dt.tz_convert(eastern)

  start_time = grouped['BEGINTIME'].iloc[0]
  end_time = grouped['timestamp'].max()
  duration = (end_time - start_time).total_seconds()/60
  duration_max = (15 * (len(grouped)+1)) + duration
  duration_mean = (duration_max + duration)/2
  customers_affected  = grouped['CUSTOMERSOUT'].mean()
  county = grouped['COMMUNITIES'].iloc[0]
  emc = grouped['EMC'].iloc[0]
  lat = grouped['y'].iloc[0]
  lon = grouped['x'].iloc[0]
  total_cust_out = customers_affected * duration_mean

  return pd.Series({
      'start_time' : start_time,
      'end_time' : end_time,
      'lat' : lat,
      'long' : lon,
      'county_name' : county,
      'utility_provider' : emc,
      'duration_diff' : duration,
      'duration_max' : duration_max,
      'duration_mean' : duration_mean,
      'customers_affected' : customers_affected,
      'total_customer_outage_time' : total_cust_out
  })



In [28]:
def groupdata(data):
  grouped = data.groupby('BATCHID')
  result = grouped.apply(compute).reset_index(drop=True)
  return result


In [29]:
groupdata(df)

,start_time,end_time,lat,long,county_name,utility_provider,duration_diff,duration_max,duration_mean,customers_affected,total_customer_outage_time
0,2023-09-30 13:59:00-04:00,2023-10-01 02:10:09-04:00,4.816313e+06,-1.169233e+07,Denver,Southwestern Public Service Co.,731.150000,2096.150000,1413.650000,15.866667,22429.913333
1,2023-07-31 06:34:00-04:00,2023-08-01 02:10:09-04:00,4.828438e+06,-1.168690e+07,Denver,Southwestern Public Service Co.,1176.150000,1941.150000,1558.650000,23.060000,35942.469000
2,2023-08-30 23:56:00-04:00,2023-09-01 02:10:09-04:00,4.819046e+06,-1.169152e+07,Denver,Southwestern Public Service Co.,1574.150000,1814.150000,1694.150000,5.400000,9148.410000
3,2023-05-31 17:19:00-04:00,2023-06-01 02:10:06-04:00,3.803233e+06,-1.158367e+07,Loving,Southwestern Public Service Co.,531.100000,1911.100000,1221.100000,28.989011,35398.481319
4,2023-04-30 19:28:00-04:00,2023-05-01 02:10:09-04:00,4.821522e+06,-1.168701e+07,Denver,Southwestern Public Service Co.,402.150000,702.150000,552.150000,23.315789,12873.813158
...,...,...,...,...,...,...,...,...,...,...,...
17929,2023-05-31 12:50:00-04:00,2023-06-01 01:40:09-04:00,4.941769e+06,-1.164257e+07,NaN,Southwestern Public Service Co.,770.150000,2075.150000,1422.650000,37.720930,53663.681395
17930,2023-07-31 13:32:00-04:00,2023-08-01 01:40:10-04:00,4.822562e+06,-1.165572e+07,Aurora,Southwestern Public Service Co.,728.166667,1523.166667,1125.666667,25.961538,29224.038462
17931,2023-08-31 17:46:00-04:00,2023-09-01 01:55:10-04:00,4.843603e+06,-1.168479e+07,Denver,Southwestern Public Service Co.,489.166667,744.166667,616.666667,5.125000,3160.416667
17932,2023-05-31 12:50:00-04:00,2023-06-01 01:55:10-04:00,4.941769e+06,-1.164257e+07,NaN,Southwestern Public Service Co.,785.166667,2135.166667,1460.166667,37.404494,54616.795880
